#### Imports

In [1]:
# 0orange 1red 2white 3yellow 4blue 5green
import cv2
import numpy as np
import keyboard
from time import time
from time import sleep

#### Cube Solving Algorithm Communication Functions (hkociemba/RubiksCube-TwophaseSolverFiveFaces)

In [2]:
# you must run the server first from this path
# RubiksCubeSolver\NemaMachine\Code\serverBatFiles\Server.bat

In [3]:
import socket
DEFAULT_HOST = 'localhost'
DEFAULT_PORT = 8080
mySocket = None

def connectToSolvingAlgServer():
    global mySocket
    try:
        mySocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    except socket.error:
        show_text('Failed to create socket')
        return False
    try:
        remote_ip = socket.gethostbyname(DEFAULT_HOST)
    except socket.gaierror:
        print("Hostname could not be resolved.")
        return False
    try:
        mySocket.connect((remote_ip, DEFAULT_PORT))
    except Exception as e:
        print('Cannot connect to server!', e)
        return False
    print('Connected with ' + remote_ip + '\n')
    return True

def sendScrambleGetSolution(scramble):
    global mySocket
    try:
        mySocket.sendall((scramble + '\n\n').encode())
    except Exception as e:
        print('Cannot send cube configuration to server.', e)
        return
    solution = mySocket.recv(2048).decode()
    return solution

#### Global Variables

In [4]:
commands = []
#troll = []

#### Serial Communication

In [5]:
from serial import Serial
from serial import EIGHTBITS
from serial import PARITY_NONE
from serial import STOPBITS_ONE

port = "COM9"
running = True
def communicationThread():
    global commands, running#, troll
    
    # init serial
    serial = Setup_serial(port)
    if serial != None:
        print("Serial opened!")
    else:
        print("Could not open serial! returning")
        running = False
        return
    
    # run communication
    keepGoing = True
    while running:
        if len(commands)>0 and keepGoing:
            keepGoing = False
            '''
            # fill the reverser array, just for test
            if commands[0].startswith("MMOVE"):
                move = commands[0].split(" ")[1]
                if move=="D'":
                    troll.append("D")
                elif move=="D":
                    troll.append("D'")

                elif move=="F'":
                    troll.append("F")
                elif move=="F":
                    troll.append("F'")

                elif move=="B'":
                    troll.append("B")
                elif move=="B":
                    troll.append("B'")

                elif move=="L'":
                    troll.append("L")
                elif move=="L":
                    troll.append("L'")

                elif move=="R'":
                    troll.append("R")
                elif move=="R":
                    troll.append("R'")
                else:
                    print("Python said: Unknown move " + str(move))
            '''
                    
            send_command(serial, commands.pop(0))

        # FUTURE: check for force reports in order for us to decide where to take the robot arm

        # check serial for msgs
        bundle = Read(serial)
        message = Clean(bundle)
        if message: # skip what's below this line

            # process msg, either it's a confirmation to a command or applied force reporting
            if message.startswith("Ok, finished the sequence"):
                keepGoing = True
            elif message.startswith("Ok, finished the manual"):
                keepGoing = True
            else:
                keepGoing = True
                print("Keep going")
                print("Arduino printed:", message)

        # whether we should stop the program or not
        cv2.waitKey(1)
        
    # close serial boi
    if serial == None:
        serial.close()
        serial = None
        
def Read(serial):
    try:
        return serial.readline() # read a byte string
    except Exception as e:
        print("Failed to read error:", e)

        
def Clean(b):
    if b!=None:
        #string_n = b.decode('utf-8', errors='replace')  # decode byte string into Unicode
        try:
            return b.decode().rstrip()  # decode byte string into Unicode
        except UnicodeDecodeError:
            return None

def Setup_serial(port):
    global running
    while running:
        cv2.waitKey(1)
        try:
            '''
            ser = Serial(
            port = port,
            baudrate = 9600,
            bytesize = EIGHTBITS,
            parity = PARITY_NONE,
            stopbits = STOPBITS_ONE,
            timeout = 1,
            xonxoff = False,
            rtscts = False,
            dsrdtr = False,
            writeTimeout = 2
            )
            '''
            ser = Serial(port = port, baudrate = 9600, timeout = 0.001)
            if(ser.isOpen() == False):
                ser.open()
            if ser != None:
                return ser
        except FileNotFoundError as e:
            print("You're either not connected or the port", port, "is wrong")
        except Exception as e:
            print("Error connecting to serial with error e=", e)
            return None
        print("Attempting to open serial")
        

def send_command(serial, command):
    serial.write(bytes(command + '\n', 'utf-8'))
    return serial


#### Keyboard Clicks

In [11]:
e_pressed = False
d_pressed = False
r_pressed = False
f_pressed = False
s_pressed = False
t_pressed = False
g_pressed = False
y_pressed = False
h_pressed = False
u_pressed = False
j_pressed = False
i_pressed = False
k_pressed = False

def keyboardCheck():
    global commands
    global k_pressed, e_pressed, d_pressed, r_pressed, f_pressed, s_pressed, t_pressed, t_pressed, g_pressed, y_pressed, h_pressed, u_pressed, j_pressed, i_pressed
    
    if keyboard.is_pressed('e') == True:
        if not e_pressed:
            commands.append("MMOVE L")
            L(True)
        e_pressed = True
    else:
        e_pressed = False

    if keyboard.is_pressed('d') == True:
        if not d_pressed:
            commands.append("MMOVE L'")
            L(False)
        d_pressed = True
    else:
        d_pressed = False



    if keyboard.is_pressed('r') == True:
        if not r_pressed:
            commands.append("MMOVE R")
            R(True)
        r_pressed = True
    else:
        r_pressed = False

    if keyboard.is_pressed('f') == True:
        if not f_pressed:
            commands.append("MMOVE R'")
            R(False)
        f_pressed = True
    else:
        f_pressed = False
        
    if keyboard.is_pressed('t') == True:
        if not t_pressed:
            commands.append("MMOVE F")
            F(True)
        t_pressed = True
    else:
        t_pressed = False

    if keyboard.is_pressed('g') == True:
        if not g_pressed:
            commands.append("MMOVE F'")
            F(False)
        g_pressed = True
    else:
        g_pressed = False


        
    if keyboard.is_pressed('y') == True:
        if not y_pressed:
            commands.append("MMOVE D")
            D(True)
        y_pressed = True
    else:
        y_pressed = False

    if keyboard.is_pressed('h') == True:
        if not h_pressed:
            commands.append("MMOVE D'")
            D(False)
        h_pressed = True
    else:
        h_pressed = False

        
    if keyboard.is_pressed('u') == True:
        if not u_pressed:
            commands.append("MMOVE B")
            B(True)
        u_pressed = True
    else:
        u_pressed = False

    if keyboard.is_pressed('j') == True:
        if not j_pressed:
            commands.append("MMOVE B'")
            B(False)
        j_pressed = True
    else:
        j_pressed = False


    '''
    if keyboard.is_pressed('i') == True:
        if not i_pressed:
            U(True)
        i_pressed = True
    else:
        i_pressed = False


    if keyboard.is_pressed('k') == True:
        if not k_pressed:
            U(False)
        k_pressed = True
    else:
        k_pressed = False
    '''

    if keyboard.is_pressed('s') == True:
        if not s_pressed:
            solve()
        s_pressed = True
    else:
        s_pressed = False

#### Gui

In [12]:
def draw(window):

    x = 0
    y = 90
    window[y:y+30, x:x+30, :] = list(get_color(cube["down"][6]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["down"][7]))
    y += 30
    window[y:y+30, x:x+30, :]= list(get_color(cube["down"][8]))
    y = 90
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["down"][3]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["down"][4]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["down"][5]))
    y = 90
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["down"][0]))
    y += 30
    window[y:y+30, x:x+30, :]= list(get_color(cube["down"][1]))
    y += 30
    window[y:y+30, x:x+30, :]= list(get_color(cube["down"][2]))



    
    x += 30
    y = 0
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][8]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][5]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][2]))
    y = 0
    x += 30
    
    window[y:y+30, x:x+30, :]= list(get_color(cube["left"][7]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][4]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][1]))
    y = 0
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][6]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][3]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["left"][0]))




    x += 30 - 30 * 3
    y = 30*3
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][6]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][7]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][8]))
    y = 30*3
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][3]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][4]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][5]))
    y = 30*3
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][0]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["front"][1]))
    y += 30
    window[y:y+30, x:x+30, :]= list(get_color(cube["front"][2]))

    

    x += 30 - 30 * 3
    y = 30*3 * 2
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][0]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][3]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][6]))
    y = 30*3 * 2
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][1]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][4]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][7]))
    y = 30*3 * 2
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][2]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][5]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["right"][8]))


    x += 30
    y = 30*3 * 1
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][6]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][7]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][8]))
    y = 30*3 * 1
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][3]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][4]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][5]))
    y = 30*3 * 1
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][0]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][1]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["up"][2]))



    x += 30
    y = 30*3 * 1
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][2]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][1]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][0]))
    y = 30*3 * 1
    x += 30

    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][5]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][4]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][3]))
    y = 30*3 * 1
    x += 30
    
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][8]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][7]))
    y += 30
    window[y:y+30, x:x+30, :] = list(get_color(cube["back"][6]))
    
    return window

#### Cube Logic

In [13]:
def resetCube():
    return {
        "left": [0, 0, 0, 0, 0, 0, 0, 0, 0],
        "right": [1, 1, 1, 1, 1, 1, 1, 1, 1],
        "up": [2, 2, 2, 2, 2, 2, 2, 2, 2],
        "down": [3, 3, 3, 3, 3, 3, 3, 3, 3],
        "back": [4, 4, 4, 4, 4, 4, 4, 4, 4],
        "front": [5, 5, 5, 5, 5, 5, 5, 5, 5]
    }
cube = resetCube()



def get_color(index):
    if index==0:
        return (0, 165, 255)
    if index==1:
        return (0, 0, 255)
    if index==2:
        return (255, 255, 255)
    if index==3:
        return (0, 255, 255)
    if index==4:
        return (255, 0, 0)
    if index==5:
        return (0, 255, 0)

def get_color_name(index):
    if index==0:
        return "L"
    if index==1:
        return "R"
    if index==2:
        return "U"
    if index==3:
        return "D"
    if index==4:
        return "B"
    if index==5:
        return "F"

def get_x_rotated_color_name(index):
    if index==0:
        return "L"
    if index==1:
        return "R"
    if index==2:
        return "B"
    if index==3:
        return "F"
    if index==4:
        return "D"
    if index==5:
        return "U"
    
def unrotateLetter(letter):
    if letter=="L":
        return "L"
    if letter=="R":
        return "R"
    if letter=="U":
        return "F"
    if letter=="D":
        return "B"
    if letter=="F":
        return "D"
    return letter
    
def turn_face(cube, cw_or_acw, face):
    if cw_or_acw:
        cube[face] = [
            cube[face][6], 
            cube[face][3], 
            cube[face][0], 
            cube[face][7], 
            cube[face][4], 
            cube[face][1], 
            cube[face][8], 
            cube[face][5],
            cube[face][2]
        ]
    else:
        cube[face] = [
            cube[face][2], 
            cube[face][5], 
            cube[face][8], 
            cube[face][1], 
            cube[face][4], 
            cube[face][7], 
            cube[face][0], 
            cube[face][3], 
            cube[face][6]
        ]
    return cube

def L(cw_or_acw):
    global cube

    # turn the face itself
    cube = turn_face(cube, cw_or_acw, "left")

    # turn surounding faces

    if cw_or_acw:
        temp_down = cube["down"].copy()
        
        cube["down"][0] = cube["front"][0] 
        cube["down"][3] = cube["front"][3]
        cube["down"][6] = cube["front"][6]

        cube["front"][0] = cube["up"][0]
        cube["front"][3] = cube["up"][3]
        cube["front"][6] = cube["up"][6]

        cube["up"][6] = cube["back"][2]
        cube["up"][3] = cube["back"][5]
        cube["up"][0] = cube["back"][8]

        cube["back"][8] = temp_down[0] 
        cube["back"][5] = temp_down[3]
        cube["back"][2] = temp_down[6]

    else:
        temp_back = cube["back"].copy()
        cube["back"][2] = cube["up"][6]
        cube["back"][5] = cube["up"][3]
        cube["back"][8] = cube["up"][0]
        
        cube["up"][0] = cube["front"][0]
        cube["up"][3] = cube["front"][3]
        cube["up"][6] = cube["front"][6]
        
        cube["front"][0] = cube["down"][0]
        cube["front"][3] = cube["down"][3]
        cube["front"][6] = cube["down"][6]
        
        cube["down"][0] = temp_back[8]
        cube["down"][3] = temp_back[5]
        cube["down"][6] = temp_back[2]
        

def R(cw_or_acw):
    global cube

    # turn the face itself
    cube = turn_face(cube, cw_or_acw, "right")

    # turn surounding faces

    if cw_or_acw:
        temp_down = cube["down"].copy()
        
        cube["down"][8] = cube["back"][0] 
        cube["down"][5] = cube["back"][3]
        cube["down"][2] = cube["back"][6]

        cube["back"][6] = cube["up"][2]
        cube["back"][3] = cube["up"][5]
        cube["back"][0] = cube["up"][8]

        cube["up"][2] = cube["front"][2]
        cube["up"][5] = cube["front"][5]
        cube["up"][8] = cube["front"][8]

        cube["front"][2] = temp_down[2] 
        cube["front"][5] = temp_down[5]
        cube["front"][8] = temp_down[8]

    else:
        temp_back = cube["back"].copy()
        cube["back"][6] = cube["down"][2]
        cube["back"][3] = cube["down"][5]
        cube["back"][0] = cube["down"][8]
        
        cube["down"][2] = cube["front"][2]
        cube["down"][5] = cube["front"][5]
        cube["down"][8] = cube["front"][8]
        
        cube["front"][2] = cube["up"][2]
        cube["front"][5] = cube["up"][5]
        cube["front"][8] = cube["up"][8]
        
        cube["up"][8] = temp_back[0]
        cube["up"][5] = temp_back[3]
        cube["up"][2] = temp_back[6]


def U(cw_or_acw):
    global cube

    # turn the face itself
    cube = turn_face(cube, cw_or_acw, "up")

    # turn surounding faces

    if cw_or_acw:
        temp_left = cube["left"].copy()
        
        cube["left"][0] = cube["front"][0] 
        cube["left"][1] = cube["front"][1]
        cube["left"][2] = cube["front"][2]

        cube["front"][0] = cube["right"][0]
        cube["front"][1] = cube["right"][1]
        cube["front"][2] = cube["right"][2]

        cube["right"][0] = cube["back"][0]
        cube["right"][1] = cube["back"][1]
        cube["right"][2] = cube["back"][2]

        cube["back"][0] = temp_left[0] 
        cube["back"][1] = temp_left[1]
        cube["back"][2] = temp_left[2]

    else:
        temp_left = cube["left"].copy()
        
        cube["left"][0] = cube["back"][0] 
        cube["left"][1] = cube["back"][1]
        cube["left"][2] = cube["back"][2]

        cube["back"][0] = cube["right"][0]
        cube["back"][1] = cube["right"][1]
        cube["back"][2] = cube["right"][2]

        cube["right"][0] = cube["front"][0]
        cube["right"][1] = cube["front"][1]
        cube["right"][2] = cube["front"][2]

        cube["front"][0] = temp_left[0] 
        cube["front"][1] = temp_left[1]
        cube["front"][2] = temp_left[2]


def D(cw_or_acw):
    global cube

    # turn the face itself
    cube = turn_face(cube, cw_or_acw, "down")

    # turn surounding faces

    if cw_or_acw:
        temp_left = cube["left"].copy()
        
        cube["left"][6] = cube["back"][6] 
        cube["left"][7] = cube["back"][7]
        cube["left"][8] = cube["back"][8]

        cube["back"][6] = cube["right"][6]
        cube["back"][7] = cube["right"][7]
        cube["back"][8] = cube["right"][8]

        cube["right"][6] = cube["front"][6]
        cube["right"][7] = cube["front"][7]
        cube["right"][8] = cube["front"][8]

        cube["front"][6] = temp_left[6] 
        cube["front"][7] = temp_left[7]
        cube["front"][8] = temp_left[8]

    else:
        temp_left = cube["left"].copy()
        
        cube["left"][6] = cube["front"][6] 
        cube["left"][7] = cube["front"][7]
        cube["left"][8] = cube["front"][8]

        cube["front"][6] = cube["right"][6]
        cube["front"][7] = cube["right"][7]
        cube["front"][8] = cube["right"][8]

        cube["right"][6] = cube["back"][6]
        cube["right"][7] = cube["back"][7]
        cube["right"][8] = cube["back"][8]

        cube["back"][6] = temp_left[6] 
        cube["back"][7] = temp_left[7]
        cube["back"][8] = temp_left[8]

def B(cw_or_acw):
    global cube

    # turn the face itself
    cube = turn_face(cube, cw_or_acw, "back")

    # turn surounding faces
    if cw_or_acw:
        temp_left = cube["left"].copy()
        
        cube["left"][6] = cube["up"][0] 
        cube["left"][3] = cube["up"][1]
        cube["left"][0] = cube["up"][2]

        cube["up"][0] = cube["right"][2]
        cube["up"][1] = cube["right"][5]
        cube["up"][2] = cube["right"][8]

        cube["right"][2] = cube["down"][8]
        cube["right"][5] = cube["down"][7]
        cube["right"][8] = cube["down"][6]

        cube["down"][6] = temp_left[0] 
        cube["down"][7] = temp_left[3]
        cube["down"][8] = temp_left[6]

    else:
        temp_left = cube["left"].copy()
        
        cube["left"][0] = cube["down"][6] 
        cube["left"][3] = cube["down"][7]
        cube["left"][6] = cube["down"][8]

        cube["down"][6] = cube["right"][8]
        cube["down"][7] = cube["right"][5]
        cube["down"][8] = cube["right"][2]

        cube["right"][2] = cube["up"][0]
        cube["right"][5] = cube["up"][1]
        cube["right"][8] = cube["up"][2]

        cube["up"][0] = temp_left[6] 
        cube["up"][1] = temp_left[3]
        cube["up"][2] = temp_left[0]

def F(cw_or_acw):
    global cube

    # turn the face itself
    cube = turn_face(cube, cw_or_acw, "front")

    # turn surounding faces
    if cw_or_acw:
        temp_up = cube["up"].copy()
        
        cube["up"][6] = cube["left"][8] 
        cube["up"][7] = cube["left"][5]
        cube["up"][8] = cube["left"][2]

        cube["left"][8] = cube["down"][2]
        cube["left"][5] = cube["down"][1]
        cube["left"][2] = cube["down"][0]

        cube["down"][2] = cube["right"][0]
        cube["down"][1] = cube["right"][3]
        cube["down"][0] = cube["right"][6]

        cube["right"][0] = temp_up[6] 
        cube["right"][3] = temp_up[7]
        cube["right"][6] = temp_up[8]

    else:
        temp_up = cube["up"].copy()
        
        cube["up"][6] = cube["right"][0]
        cube["up"][7] = cube["right"][3]
        cube["up"][8] = cube["right"][6]

        cube["right"][0] = cube["down"][2]
        cube["right"][3] = cube["down"][1]
        cube["right"][6] = cube["down"][0]

        cube["down"][2] = cube["left"][8]
        cube["down"][1] = cube["left"][5]
        cube["down"][0] = cube["left"][2]

        cube["left"][8] = temp_up[6] 
        cube["left"][5] = temp_up[7]
        cube["left"][2] = temp_up[8]

def x_turn(rotated_cube):
    
    rotated_cube = turn_face(rotated_cube, False, "left")
    rotated_cube = turn_face(rotated_cube, True, "right")
    backup_back = rotated_cube["back"].copy()
    rotated_cube["back"] = [
        rotated_cube["up"][8],
        rotated_cube["up"][7],
        rotated_cube["up"][6],
        rotated_cube["up"][5],
        rotated_cube["up"][4],
        rotated_cube["up"][3],
        rotated_cube["up"][2],
        rotated_cube["up"][1],
        rotated_cube["up"][0]
    ]
    rotated_cube["up"] = [
        rotated_cube["front"][0],
        rotated_cube["front"][1],
        rotated_cube["front"][2],
        rotated_cube["front"][3],
        rotated_cube["front"][4],
        rotated_cube["front"][5],
        rotated_cube["front"][6],
        rotated_cube["front"][7],
        rotated_cube["front"][8]
    ]
    rotated_cube["front"] = [
        rotated_cube["down"][0],
        rotated_cube["down"][1],
        rotated_cube["down"][2],
        rotated_cube["down"][3],
        rotated_cube["down"][4],
        rotated_cube["down"][5],
        rotated_cube["down"][6],
        rotated_cube["down"][7],
        rotated_cube["down"][8]
    ]
    rotated_cube["down"] = [
        backup_back[8],
        backup_back[7],
        backup_back[6],
        backup_back[5],
        backup_back[4],
        backup_back[3],
        backup_back[2],
        backup_back[1],
        backup_back[0]
    ]
    return rotated_cube
        
    
    

#### Cube Solver

In [14]:
'''
#import kociemba
def cleanMove(troll, letter):
    # clean it out exactly in this order otherwise the ' will get mixed up
    # remove doubles
    needsSimplifying = False
    while troll.find(letter + letter + "'")>-1: # ex. replace all FF' by nothing
        needsSimplifying = True
        troll = troll.replace(letter + letter + "'", "")
    while troll.find(letter + "'" + letter + "'")>-1: # ex. replace all F'F' by F2
        needsSimplifying = True
        troll = troll.replace(letter + "'" + letter + "'", letter + "2")
    while troll.find(letter + "'" + letter)>-1:   # ex. replace all F'F by nothing
        needsSimplifying = True
        troll = troll.replace(letter + "'" + letter, "")
    while troll.find(letter + letter)>-1:     # ex. replace all FF by F2
        needsSimplifying = True
        troll = troll.replace(letter + letter, letter + "2")
        
    # remove double doubles
    while troll.find(letter + "2" + letter + "2")>-1:     # ex. replace all F2F2 by nothing
        needsSimplifying = True
        troll = troll.replace(letter + "2" + letter + "2", "")
    return troll, needsSimplifying
'''

def mover(Letter, troll, index, trollLen_1):
    if index < trollLen_1:
        if troll[index+1] == "'":
            Letter(False)
            return index + 1
        elif troll[index+1] =="2":
            Letter(True)
            Letter(True)
            return index + 1
    Letter(True)
    return index

def simulator_turn_face(letterStr, cw_or_acw):
    if letterStr == "B":
        B(cw_or_acw)
    elif letterStr == "F":
        F(cw_or_acw)
    elif letterStr == "R":
        R(cw_or_acw)
    elif letterStr == "L":
        L(cw_or_acw)
    elif letterStr == "D":
        D(cw_or_acw)
    else:
        print("Unrecognized face name", letterStr)

mimiquing_unrotated_solution = ""
mimique_index = 0
unrotated_solutionLen_1 = 0
def mimique_real_solution_in_simulation():
    global mimique_index, mimiquing_unrotated_solution
    
    if mimiquing_unrotated_solution == "":
        return
    
    # make a move in the simulation to following the real robot in real time
    if mimique_index <= unrotated_solutionLen_1:
        if mimique_index < unrotated_solutionLen_1:
            if mimiquing_unrotated_solution[mimique_index+1] == "2":
                simulator_turn_face(mimiquing_unrotated_solution[mimique_index], True)
                simulator_turn_face(mimiquing_unrotated_solution[mimique_index], True)
                mimique_index+=1
            elif mimiquing_unrotated_solution[mimique_index+1] == "'":
                simulator_turn_face(mimiquing_unrotated_solution[mimique_index], False)
                mimique_index+=1
            else:
                simulator_turn_face(mimiquing_unrotated_solution[mimique_index], True)
        else:
            simulator_turn_face(mimiquing_unrotated_solution[mimique_index], True)
        mimique_index+=1
    else:
        mimiquing_unrotated_solution = ""
        
def solve():
    #global troll
    global mimiquing_unrotated_solution, unrotated_solutionLen_1, mimique_index
    global commands
    global cube
    
    '''
    ogscramble = ""
    # convert indices to letters (ex. L, R, U, D, B)
    for piece in cube["up"]:
        ogscramble += get_color_name(piece)
    
    for piece in cube["right"]:
        ogscramble += get_color_name(piece)
    
    for piece in cube["front"]:
        ogscramble += get_color_name(piece)
    
    for piece in cube["down"]:
        ogscramble += get_color_name(piece)
    
    for piece in cube["left"]:
        ogscramble += get_color_name(piece)
    
    for piece in cube["back"]:
        ogscramble += get_color_name(piece)
    '''
        
    # rotate cube by an x to give it to hkociemba/RubiksCube-TwophaseSolverFiveFaces solving algorithm since it only doesn't do B face but ur machine doesn't do Up face
    rotated_cube = cube.copy()
    rotated_cube = x_turn(rotated_cube)
    scramble = ""

    # convert indices to letters (ex. L, R, U, D, B)
    for piece in rotated_cube["up"]:
        scramble += get_x_rotated_color_name(piece)
    
    for piece in rotated_cube["right"]:
        scramble += get_x_rotated_color_name(piece)
    
    for piece in rotated_cube["front"]:
        scramble += get_x_rotated_color_name(piece)
    
    for piece in rotated_cube["down"]:
        scramble += get_x_rotated_color_name(piece)
    
    for piece in rotated_cube["left"]:
        scramble += get_x_rotated_color_name(piece)
    
    for piece in rotated_cube["back"]:
        scramble += get_x_rotated_color_name(piece)
        
    # request solution from cube solving alg server
    #print("ogscramble", ogscramble)
    print("Rotated scramble", scramble)
    startTime = time()
    solution = sendScrambleGetSolution(scramble)
    print("Obtaining a solution took", time()-startTime, "s")
    startTime = time()
    if solution == None:
        print("Couldn't obtain solution from cube solving alg server")
    else:
        print("Received solution", solution)
        
        try:
            solution = solution.split("(")[0].replace(" ", "").replace("3", "'").replace("1", "")
                    
        except Exception as e:
            print("Couldn't process solution because of", e)
        
        # unrotate the letters
        unrotated_solution = ""
        for i in range(0, len(solution)):
            unrotated_solution += unrotateLetter(solution[i])
            
        # send solution to robot
        print("Processing it took", time()-startTime, "s")
        #print("unrotated_solution", unrotated_solution)
        commands.append("SEQUENCE " + unrotated_solution)
        
        # follow the same solution for the simulation
        mimiquing_unrotated_solution = unrotated_solution
        unrotated_solutionLen_1 = len(unrotated_solution)-1
        mimique_index = 0
    
    '''
    # clean reversed sequence, just for test
    troll.reverse()
    trollStr = ""
    for move in troll:
        trollStr += move
    #exampleTroll = "R'R'R'RRF'FR'RRL'L'LLLL'R'LL'RF'F'FF'FFFFF'F'F'F'F'FFF'F'F'"
    needsSimplifying = True #u need a whole while loop bcz while for example cleaning the 3rd letter, maybe the first two get a chance to be simplified
    while needsSimplifying:
        trollStr, one = cleanMove(trollStr, "F")
        trollStr, two = cleanMove(trollStr, "D")
        trollStr, three = cleanMove(trollStr, "R")
        trollStr, four = cleanMove(trollStr, "L")
        trollStr, five = cleanMove(trollStr, "B")
        #trollStr, five = cleanMove(trollStr, "U")
        needsSimplifying = one or two or three or four or five
    
    trollLen_1 = len(trollStr)-1
    index = 0
    while index <= trollLen_1:
        if trollStr[index]=="L":
            index = mover(L, trollStr, index, trollLen_1)
        elif trollStr[index]=="R":
            index = mover(R, trollStr, index, trollLen_1)
        #elif trollStr[index]=="U":
        #    index = mover(U, trollStr, index, trollLen_1)
        elif trollStr[index]=="D":
            index = mover(D, trollStr, index, trollLen_1)
        elif trollStr[index]=="B":
            index = mover(B, trollStr, index, trollLen_1)
        elif trollStr[index]=="F":
            index = mover(F, trollStr, index, trollLen_1)
        else:
            print("OOF, Unknown move " + str(trollStr[index]))
        index += 1
            
    print("Sending sequence:", trollStr)
    '''
    
    '''
    # Send reversed sequence one move at a time, just test
    trollStrLen_1 = len(trollStr)-1
    index = 0
    while index <= trollStrLen_1:
        if index < trollStrLen_1:
            if trollStr[index+1] == "'":
                commands.append("SEQUENCE " + trollStr[index] + "'")
                index += 1
            elif trollStr[index+1] == "2":
                commands.append("SEQUENCE " + trollStr[index] + "2")
                index += 1
            else:
                commands.append("SEQUENCE " + trollStr[index])
        else:
            commands.append("SEQUENCE " + trollStr[index])
        index += 1
    '''
    '''
    # send sequence all at once
    commands.append("SEQUENCE " + trollStr)
    troll = []
    '''


#### Main

In [15]:
cv2.destroyAllWindows()
running = True
#troll = []
cube = resetCube()
window = np.zeros(( 30*3*3, 30*4*3, 3), dtype=np.uint8)

# connect to cube machine
from threading import Thread
thread = Thread(target=communicationThread)
thread.start()

# connect to hkociemba/RubiksCube-TwophaseSolverFiveFaces
connected = connectToSolvingAlgServer()
if not connected:
    print("Couldn't connect to cube solving alg server")
else:
    print("Connected to cube solving alg server")

while running:
    
    keyboardCheck()
    mimique_real_solution_in_simulation()
        
    window = draw(window)

    cv2.imshow("window", window)
    cv2.waitKey(1)
    
    if keyboard.is_pressed("esc"):
        running = False
        if thread.is_alive():
            thread.join()
        break
        
print("\nCode finished")
cv2.destroyAllWindows()

Connected with 127.0.0.1

Connected to cube solving alg server
Serial opened!
Keep going
Arduino printed: Ok, Ready
Rotated scramble UDRRUULBDBLFBRRUFBFLLLFRBDLRLFDDUBDLRUDBLFRFUDRFFBUUBD
Obtaining a solution took 0.016701221466064453 s
Received solution U3 F1 D1 F2 R3 U3 D1 L1 U3 D2 L3 U2 D3 F2 R2 D2 R2 L2 F2 R2 D1 R2 D1 (23f)

Processing it took 0.0 s
Keep going
Arduino printed: R'F'BLF'B2L'F2B'D2R2B2R2L2D2R2BR2B

Code finished
